In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from datetime import datetime
from datetime import timedelta

import warnings
warnings.filterwarnings(action='ignore') 

# 사이킷런
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [2]:
pd.set_option('display.max_row', 5000)
pd.set_option('display.max_columns', 100)

from matplotlib import font_manager, rc
import matplotlib
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()

rc('font', family=font_name)

matplotlib.rcParams['axes.unicode_minus'] = False

# 지연변수 만들기
* 각 농가별 주차별로
* 농가 모두 합쳐서 전체 표본을 대상으로
</br>
</br>지연 시간은 5주 ~ 10주까지 돌려보자
1. 생장길이
2. 경경
3. 화방높이

----
# 사용자 지정함수

In [3]:
# 생육 데이터 로드
def sang_data_load(sang_farm_name):
    n = 'C:\\농업기술원\\1차\\2018\\2018-2019 토마토_생육(원본)/'+sang_farm_name+'.xlsx'
    col = ['농가명', 'Date', 'Year', 'Month', 'Day', 'Week', 'WeekNum', 'Sample', '생장길이', '화방높이', '줄기굵기']
    sang = pd.read_excel(n)
    sang = sang[col]
    return sang

In [4]:
# 환경 데이터 로드
def env_data_load(env_farm_name):
    n = 'C:\\농업기술원\\1차\\2018\\2018-2019 토마토_환경(원본)/'+env_farm_name+'.xlsx'
    #col = ['date', '내부온도', '내부습도', '지온', '지습', '이슬점', 'CO2', '외부온도', '풍속', '일사량', '누적일사량','강우', '포화수분', '절대습도', '수분부족분', '이중창환기온도', '환기온도', '난방온도']
    col = ['date', '내부온도', '내부습도', 'CO2', '누적일사량']
    env = pd.read_excel(n)
    env = env[col]
    return env

In [5]:
# 주차 추가
def add_weeknum(sang_df):
    sang_df['diff']=sang_df['WeekNum'].shift(1)
    sang_df['diff2']=sang_df['WeekNum']-sang_df['diff']
    
    num=-1
    test_week=[]
    for i in range(len(sang_df)):
        if sang_df['diff2'].loc[i]!=0:
            num+=1
            test_week.append(num)
        else:
            test_week.append(num)
    del sang_df['diff']
    del sang_df['diff2']
    return test_week

In [6]:
# 12시간 짤라서 Date 추가
def env_add_Date(env_df,생육조사기간):
    df=pd.DataFrame()
    for i in range(len(생육조사기간)-1):
        start_i = 생육조사기간[i]+timedelta(hours=12)
        end_i = 생육조사기간[i+1]+timedelta(hours=12)

        df2 = env_df[(env_df['date']>=start_i) & (env_df['date']<end_i)]
        df2['Date']=생육조사기간[i+1]


        df=pd.concat([df,df2])
    return df

In [7]:
def cumsum_to_value(df):
    #누적일사량 해결
    df['lag_누적일사량']=df['누적일사량'].shift(1)
    df['일사량2']=df['누적일사량']-df['lag_누적일사량']

    #음수를 0으로 치환 NA를 0으로 치환
    df['일사량2_치환']=df['일사량2'].apply(lambda x : 0 if x<0 else x)
    df['일사량']=df['일사량2_치환'].fillna(0)

    del df['lag_누적일사량']
    del df['일사량2']
    del df['일사량2_치환']
    return df

In [7]:
# 생육 Date와 환경 Date 매칭
def match_test_weeknum(sang_data,env_data):
    dic=dict(zip(sang_data['Date'],sang_data['test_weeknum']))
    result=env_data.replace({"Date":dic})["Date"]
    return result

In [7]:
# 아래 함수에 대한 pivot_table
def my_pivot(df,type=None):
    if type=='avg':
        result = df.pivot_table(index="주차",
                               values=['내부온도','내부습도','CO2'],
                               aggfunc=type)
        
    if type=='cumsum':
        
        

In [29]:
# 1주일 단위로 각 변수 집계
def cal_col(env_data,avg_col_list,cumsum_col_list,sum_col_list):
    # 평균 DF
    avg_df=env_data[avg_col_list]
        
    # 누적합 DF
    cumsum_df=env_data[cumsum_col_list]
    
    # 합 DF
    sum_df
    

In [11]:
env_data.head(5)

,date,내부온도,내부습도,CO2,누적일사량,Date,주차
35264,2018-11-19 12:00:00,23.5,60.4,282,525,2018-11-26,1
35265,2018-11-19 12:01:01,23.4,58.9,287,528,2018-11-26,1
35266,2018-11-19 12:02:00,23.4,58.6,290,531,2018-11-26,1
35267,2018-11-19 12:03:00,23.4,58.9,283,535,2018-11-26,1
35268,2018-11-19 12:04:00,23.3,59.0,287,538,2018-11-26,1


In [ ]:
# shift n주차
def last_week(n):
    # 표본 넘버,날짜로 정렬하고 쉬프트 시키면 끝일듯?
    # 쉬프트 시켜서 다음 표본의 첫번째 행은 NA로 치환
    

----
# 1. 생장길이

In [10]:
# 한 농장 sang 로드
sang_data = sang_data_load('민수네토마토_강윤성_생육')

# 한 농장 env 로드
env_data = env_data_load('민수네토마토_강윤성_환경')

# 한 농장에 주차 추가
sang_data['test_weeknum']=add_weeknum(sang_data)

# 환경에 Date컬럼 추가, 12시기준 쪼개기
생육조사기간=sorted(list(set(list(sang_data['Date']))))
env_data=env_add_Date(env_data,생육조사기간)

# 환경 데이터에 주차 컬럼 추가
env_data['주차']=match_test_weeknum(sang_data,env_data)

# 누적 해결
base_col=['date','Date','주차']
cumsum_col=['누적일사량']
cumsum_col=base_col.append(sumsum_col)

cumsum_df=cumsum_to_value(env_data[cumsum_col])


# 일주일단위로 집계
avg_list=['date','내부온도','내부습도','CO2','Date','주차']
cumsum_list=['date','누적일사량','Date','주차']
sum_list=[]
env_data_week=cal_col(env_data,avg_list,sum_list,cumsum_list)